instructions: run entire notebook after specifying directory variable at the top
> model = resnet152

In [1]:
directory = "/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_v2"
directory2 = "/home/paperspace/Documents/dc-grocery-outlet-brand/image_splits_10_or_less"

In [2]:
#torchvision.__version__

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils import data
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import random
from model import *
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image

import pprint, pickle


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

/home/paperspace/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def list_files(d):
    r = []
    for root, dirs, files in os.walk(d):
        for name in files:
            r.append(os.path.join(root, name))
    return r

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# model = models.resnet34(pretrained=True)
model = models.resnet152(pretrained=True) # we do not specify pretrained=True, i.e. do not load default weights
# model.load_state_dict(torch.load('XNAS/trained_models/xnas_small_cifar10.t7'), strict=False)

for param in model.parameters():
    param.requires_grad = False
    
num_ftrs = model.fc.in_features
print(get_out_features())
model.fc = nn.Linear(num_ftrs,83)
model = model.to(device)

cuda:0
2369


In [10]:
df = pd.read_csv('UPC-Images-Sample-Labeled.csv').drop(['Short Description', 'ItemNumber', 'SubClass', 'Department', 'class_subclass', 'label'], axis = 1)
df.Brand = df.Brand.str.lower()
df = df.sort_values(by='Brand', ascending=True)
no_dupes_df = df.drop_duplicates("ImageList")
brand_df = no_dupes_df[no_dupes_df.Brand.notna()]
# brands_10_or_less = brand_df.groupby('Brand').filter(lambda subdf: len(subdf) <= 10)['Brand']

# more_than_10_df = brand_df[~brand_df['Brand'].isin(brands_10_or_less)]
#more_than_10_df.Brand = more_than_10_df.Brand.str.lower()
#more_than_10_df.Brand = more_than_10_df.Brand.str.replace(' ', '')

class_to_num = dict(zip(np.unique(brand_df['Class'].values), np.arange(len(np.unique(brand_df['Class'].values)))))

In [7]:
#more_than_10_df[more_than_10_df['Brand'] == 'reach']

In [7]:
def read_make_image_splits(file):
    """
        given a .pkl file, returns a 3D numpy array
    """

    pkl_file = open(file, 'rb')

    data1 = pickle.load(pkl_file)
    pkl_file.close()

    return data1



In [8]:
def test_model(model, generator):
    pbar = tqdm(val_generator)
    running_corrects = 0
    for step, batch in enumerate(pbar):
        model.eval()
      # inputs = inputs.to(device)
      # labels = labels.to(device)
        inputs, labels = batch
        inputs = inputs.to(torch.float)
        inputs = inputs.to(device)
        labels = labels.to(device)

      # zero the parameter gradients
        optimizer.zero_grad()

      # forward
      # track history if only in train
      # with torch.set_grad_enabled(phase == 'train'):
      
    
      
      #print(tuple(images)[0])
      # print(labels)
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            # print(preds)
      
    epoch_acc = running_corrects / len(val)
    return epoch_acc.item()

## train the model

In [9]:
torchvision.transforms.ToTensor()

ToTensor()

In [11]:
def preprocess(file):
    """
        given a file, returns generators for training, test, and validation
    """
    
    def read_make_image_splits(file):
        """
            given a .pkl file, returns a 3D numpy array
        """

        pkl_file = open(file, 'rb')

        data1 = pickle.load(pkl_file)
        pkl_file.close()

        return data1
    
    df = pd.DataFrame(read_make_image_splits(file), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
    df = df.dropna()
    df['class'] = df[['class']].replace(class_to_num)
    df_brand = df[["image_tensor", "class"]]
    # df_brand["image_tensor"] = df_brand["image_matrix"].map(lambda img: transforms.Resize((224, 224))(torch.permute(torch.tensor(img), (2, 0, 1))))
    # df_brand.drop("image_matrix", axis=1, inplace=True)
    
    # ske = LabelEncoder()
    # df_brand.brand = df_brand.brand.str.lower()
    # df_brand.brand = df_brand.brand.str.replace(' ', '')
    #df_brand["class"] = measure_to_num
    # df_brand.brand = df_brand["brand"].astype(int)
    
    # print(len(df_brand.brand.unique()))
    train = list(map(list, zip(df_brand["image_tensor"].values, df_brand["class"].values)))
    
    random.shuffle(train)
    val_split = int(len(train)*0.8)
    test_split = int(len(train)*0.9)
    val = train[val_split:test_split]
    test = train[test_split:]
    train = train[:val_split]
    # print(train[0])
    # print(train[0][0])
    # train, val = train_test_split(train, test_size=0.2)
    # train, test = train_test_split(train, test_size=0.1)

    train_generator = data.DataLoader(train, batch_size=64)
    test_generator = data.DataLoader(test, batch_size=64)
    val_generator = data.DataLoader(val, batch_size=64)
    
    return train_generator, test_generator, val_generator, val

In [12]:
def save_checkpoint(model, optimizer, save_path, epoch):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch' : epoch
    }, save_path)

In [13]:
def load_checkpoint(model, optimizer, load_path):
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    return model, optimizer, epoch

In [ ]:
num_epochs = 1000
best_acc = 0
losses = []
scores = []
best_model_wts = None
model_path = 'class_model.pt'
start = time.time()
files = list_files(directory) + list_files(directory2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# model, optimizer = load_checkpoint(model, optimizer, model_path)

for epoch in range(num_epochs):
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    # model, optimizer = load_checkpoint(model, optimizer, model_path)
    file_num = 0
    
    for file in files:
        print('File {}/{}'.format(file_num, len(files)))
        print('-' * 10)
        
        file_num += 1
        try:
            train_generator, test_generator, val_generator, val = preprocess(file)

            pbar = tqdm(train_generator)
            running_avg_loss = 0
            for step, batch in enumerate(pbar):
                model.train()
                inputs, labels = batch
                inputs = inputs.to(torch.float)
                inputs = inputs.to(device)
                labels = labels.to(device)


                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                # with torch.set_grad_enabled(phase == 'train'):
                '''images = []
                for image_path in inputs:
                    img = Image.open(os.path.join(image_dir, image_path))
                    images.append(transforms.ToTensor()(img))'''

                #print(tuple(images)[0])
                # print(labels)

                outputs = model(inputs)

                # _, preds = torch.max(outputs, 1)
                # print(labels)
                # print(outputs)
                # print(outputs.shape)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                # print(loss)
                # backward + optimize only if in training phase
                loss.backward()
                # print(loss)
                # pbar.set_description(str(loss.item()))
                optimizer.step()
                # avg_loss = (avg_loss + loss.item()) / (step + 1)
                running_avg_loss += loss.item()
                pbar.set_description("Avg Loss: " + str(running_avg_loss / (step + 1)))

                # statistics
                # running_loss += loss.item() * inputs.size(0)
                # running_corrects += torch.sum(preds == labels.data)
                # scheduler.step()

                 #NOTE: need to fix validation because this val_generator only covers 1 file, not all 50   
        except(RuntimeError):
            print('number of dims dont match in permute')
            continue
            
        epoch_acc = test_model(model, val_generator)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        print("validation accuracy:", epoch_acc)
        scores.append(epoch_acc)
        #losses.append(running_avg_loss / len(train_generator))
        print()
        
    save_checkpoint(model, optimizer, model_path, epoch)


File 0/100
----------


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  4.62it/s]


validation accuracy: 0.45744678378105164

File 1/100
----------


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  4.96it/s]


validation accuracy: 0.4095744490623474

File 2/100
----------


...

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  5.03it/s]


validation accuracy: 0.43617019057273865

File 11/100
----------


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  4.53it/s]


validation accuracy: 0.41489359736442566

File 12/100
----------


Avg Loss: 3.701427698135376:  29%|████▍          | 7/24 [00:02<00:05,  3.00it/s]

In [ ]:
end = time.time()
print(f"The model took {(end - start) // 60} minutes to train.")

In [ ]:
ex = preprocess("image_splits_v2/split_0.pkl")
tqdm(ex[0])

In [ ]:
# d = dict(enumerate(k.flatten(), 1))
# d = str(d) ## dump as string  (pickle and other packages parse the dump as bytes)

m = ast.literal_eval(example1) ### convert the dict as str to  dict

m = np.fromiter(m.values(), dtype=float) ## convert m to nparray

In [ ]:
a

In [ ]:
np.fromstring(example1)